# 아이템기반 최근접 이웃 협업 필터링
콘텐츠기반 추천시스템과는 달리 '평점을 매긴 사용자'와 '평점' 행렬 데이터가 필요하기 때문에 MovieLens 데이터셋 사용    
dataset: https://grouplens.org/datasets/movielens/latest/    
(ml-latest-small.zip(size: 1MB)사용)

In [2]:
import pandas as pd
import numpy as np

movies = pd.read_csv('./ml-latest-small/movies.csv')
ratings = pd.read_csv('./ml-latest-small/ratings.csv')
print(movies.shape)
print(ratings.shape)

(9742, 3)
(100836, 4)


movies.csv는 movieId, title, genres 가 들어있고,    
ratings.csv는 userId, movieId, rating, timestamp 로 구성되어있다.    

사용자-영화평점 행렬을 만들기 위해 DataFrame의 pivot_table() 함수를 사용하여 변경한다.    
(pivot_table()함수의 columns파라미터에 ratings 데이터의 원하는 속성을 넣으면 movieId의 값이 columns로 변환된다.)

In [3]:
ratings = ratings[['userId', 'movieId', 'rating']]
ratings_matrix = ratings.pivot_table('rating', index='userId', columns='movieId')
ratings_matrix.head(3)

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,NaN,4.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


movieId는 어떤 영화인지 확인할 수 없어서 columns를 title로 변경하겠다.    
하지만 title 속성은 ratings파일이 아닌 movies 파일에 있으므로 해당 파일을 join해서 pivot_table에 사용하도록 하겠다.

In [5]:
# title 컬럼을 얻기 이해 movies 와 조인 수행
rating_movies = pd.merge(ratings, movies, on='movieId')

# columns='title' 로 title 컬럼으로 pivot 수행. 
ratings_matrix = rating_movies.pivot_table('rating', index='userId', columns='title')

# NaN 값을 모두 0 으로 변환 -> 평점 데이터의 최소값이 0.5 최대값이 5이므로 0을 사용하여도 무방하다
ratings_matrix = ratings_matrix.fillna(0)
ratings_matrix.head(3)

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
# 행을 기준으로 유사도를 측정하는 '코사인유사도'를 사용하기 위해 전치행렬로 변환
ratings_matrix_T = ratings_matrix.transpose()
ratings_matrix_T.head(3)

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
title,,,,,,,,,,,,,,,,,,,,,
'71 (2014),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
'Hellboy': The Seeds of Creation (2004),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'Round Midnight (1986),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
from sklearn.metrics.pairwise import cosine_similarity
# 코사인 유사도
item_sim = cosine_similarity(ratings_matrix_T, ratings_matrix_T)
# 코사인 유사도의 반환값은 numpy.ndarray이다. 이 값에 head를 사용할 수 없기 때문에 보기 편한 DataFrame 형태의 데이터로 변환하여 출력하겠다.
# cosine_similarity() 로 반환된 넘파이 행렬을 영화명을 매핑하여 DataFrame으로 변환
item_sim_df = pd.DataFrame(data=item_sim, index=ratings_matrix.columns,
                          columns=ratings_matrix.columns)
print(item_sim_df.shape)
item_sim_df.head(3)

(9719, 9719)


title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
title,,,,,,,,,,,,,,,,,,,,,
'71 (2014),1.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.141653,0.0,...,0.0,0.342055,0.543305,0.707107,0.0,0.0,0.139431,0.327327,0.0,0.0
'Hellboy': The Seeds of Creation (2004),0.0,1.000000,0.707107,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0
'Round Midnight (1986),0.0,0.707107,1.000000,0.0,0.0,0.0,0.176777,0.0,0.000000,0.0,...,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0


In [14]:
# 영화 "Godfather"와 유사한 영화 6개를 유사도가 높은 순으로 출력
item_sim_df["Godfather, The (1972)"].sort_values(ascending=False)[:6]

title
Godfather, The (1972)                        1.000000
Godfather: Part II, The (1974)               0.821773
Goodfellas (1990)                            0.664841
One Flew Over the Cuckoo's Nest (1975)       0.620536
Star Wars: Episode IV - A New Hope (1977)    0.595317
Fargo (1996)                                 0.588614
Name: Godfather, The (1972), dtype: float64

In [15]:
# 영화 "Inception"과 유사한 영화 5개를 유사도가 높은 순으로 출력(0은 제외; 0: Inception 자기자신)
item_sim_df["Inception (2010)"].sort_values(ascending=False)[1:6]

title
Dark Knight, The (2008)          0.727263
Inglourious Basterds (2009)      0.646103
Shutter Island (2010)            0.617736
Dark Knight Rises, The (2012)    0.617504
Fight Club (1999)                0.615417
Name: Inception (2010), dtype: float64

### 아이템 기반의 협업 필터링을 사용한 추천 시스템 구현
아이템 기반의 협업 필터링으로 예측 평점을 구하는 수식에서 N개를 지정하지 않고 '모든 데이터'를 비교하여 예측평점을 구하는 predic_rating() 함수를 구현해보자.    
위의 수식을 예로 들어 설명하자면,    
영화 "인셉션"에 대한 사용자1의 평점을 예측한다고 가정하자.    
"인셉션"과 가장 유사도가 높은 순의 영화와 그 영화에 대해 사용자가 실제로 입력한 영화평점을 곱한 후, 영화의 유사도를 합한 값으로 나누는 것이다.

In [20]:
def predict_rating(ratings_arr, item_sim_arr ):
    '''
    1. dot함수를 사용해서 ratings_arr(사용자가 실제 입력한 영화 평점 값)과 item_sim_arr(영화 유사도 값)을 곱한 뒤,
    2. item_sim_arr(영화 유사도 값)을 모두 더한 후
    3. 두 개의 값을 나눠준다.
    '''
    ratings_pred = ratings_arr.dot(item_sim_arr)/ np.array([np.abs(item_sim_arr).sum(axis=1)])
    return ratings_pred

In [21]:
ratings_pred = predict_rating(ratings_matrix.values , item_sim_df.values)

# 확인하기 위해 DataFrame으로 변환해준다.
ratings_pred_matrix = pd.DataFrame(data=ratings_pred, index= ratings_matrix.index,
                                   columns = ratings_matrix.columns)
ratings_pred_matrix.head(3)

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
1,0.070345,0.577855,0.321696,0.227055,0.206958,0.194615,0.249883,0.102542,0.157084,0.178197,...,0.113608,0.181738,0.133962,0.128574,0.006179,0.212070,0.192921,0.136024,0.292955,0.720347
2,0.018260,0.042744,0.018861,0.000000,0.000000,0.035995,0.013413,0.002314,0.032213,0.014863,...,0.015640,0.020855,0.020119,0.015745,0.049983,0.014876,0.021616,0.024528,0.017563,0.000000
3,0.011884,0.030279,0.064437,0.003762,0.003749,0.002722,0.014625,0.002085,0.005666,0.006272,...,0.006923,0.011665,0.011800,0.012225,0.000000,0.008194,0.007017,0.009229,0.010420,0.084501


ratings_matrix(사용자가 실제로 평점을 남긴 데이터)와 ratings_pred_matirx(아이템기반의 협업필터링을 사용해서 구한 예측 평점 데이터값)은 차이가 있다는 것을 알 수 있다.    
(Three Amigos 영화의 사용자1 평점을 보면 확실히 알 수 있음)    
    
현재 발생한 오차의 원인은 아이템 기반의 협업 필터링을 사용할 때, 행렬의 곱을 dot함수를 사용하여 내적을 통한 곱셈이 진행되는데 사용자가 실제 기입한 데이터 행렬(ratings_matrix)에는 0이 많이 들어가 있어 예측한 결과값에 영향을 크게 주는 것이다.    
    
따라서 크게 발생한 오차의 값을 줄이도록 개선해야한다.    
먼저, 현재 예측한 데이터 행렬과 실제 데이터 행렬의 오차 값을 확인하기 위해 get_mse() 함수를 작성해보겠다.    
실제 데이터 값과 예측한 데이터 값의 오차를 먼저 확인해보기 위해, 예측한 데이터 행렬과 실제 데이터 행렬에서 0을 제외한 값으로 계산해서 비교해 보도록 하겠다.

In [22]:
from sklearn.metrics import mean_squared_error

# 사용자가 평점을 부여한 영화에 대해서만 예측 성능 평가 MSE 를 구함. 
def get_mse(pred, actual):
    # Ignore nonzero terms.
    pred = pred[actual.nonzero()].flatten()
    actual = actual[actual.nonzero()].flatten()
    return mean_squared_error(pred, actual)

print('아이템 기반 모든 인접 이웃 MSE: ', get_mse(ratings_pred, ratings_matrix.values ))

아이템 기반 모든 인접 이웃 MSE:  9.895354759094706


위에서 사용했던 pred_rating()함수는 아이템 기반의 협업 필터링 수식의 N의 갯수를 지정하지 않았으므로 이번엔 N을 지정해서 예측 데이터 행렬을 생성해보겠다.

In [23]:
def predict_rating_topsim(ratings_arr, item_sim_arr, n=20):
    # 사용자-아이템 평점 행렬 크기만큼 0으로 채운 예측 행렬 초기화
    pred = np.zeros(ratings_arr.shape)

    # 사용자-아이템 평점 행렬의 열 크기만큼 Loop 수행. 
    for col in range(ratings_arr.shape[1]):
        # 유사도 행렬에서 유사도가 큰 순으로 n개 데이터 행렬의 index 반환
        top_n_items = [np.argsort(item_sim_arr[:, col])[:-n-1:-1]]
        # 개인화된 예측 평점을 계산
        for row in range(ratings_arr.shape[0]):
            pred[row, col] = item_sim_arr[col, :][top_n_items].dot(ratings_arr[row, :][top_n_items].T) 
            pred[row, col] /= np.sum(np.abs(item_sim_arr[col, :][top_n_items]))        
    return pred

In [24]:
ratings_pred = predict_rating_topsim(ratings_matrix.values , item_sim_df.values, n=20)
print('아이템 기반 인접 TOP-20 이웃 MSE: ', get_mse(ratings_pred, ratings_matrix.values ))


# 계산된 예측 평점 데이터는 DataFrame으로 재생성
ratings_pred_matrix = pd.DataFrame(data=ratings_pred, index= ratings_matrix.index,
                                   columns = ratings_matrix.columns)

아이템 기반 인접 TOP-20 이웃 MSE:  3.6949999176225483


In [26]:
ratings_pred_matrix.head(3)

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.220798,0.0,0.0,1.677291,0.284372
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000


오차가 비교적 많이 줄어든 ratings_pred_matrix를 사용해 사용자1번(userId=1)에게 영화를 추천해보겠다.    
사용자1이 좋아하는 영화 순서를 확인해보자.

In [27]:
user_rating_id = ratings_matrix.loc[1, :]
user_rating_id[ user_rating_id > 0].sort_values(ascending=False)[:10]

title
Go (1999)                                                5.0
Indiana Jones and the Temple of Doom (1984)              5.0
Star Wars: Episode VI - Return of the Jedi (1983)        5.0
Star Wars: Episode V - The Empire Strikes Back (1980)    5.0
Star Wars: Episode IV - A New Hope (1977)                5.0
Spaceballs (1987)                                        5.0
South Park: Bigger, Longer and Uncut (1999)              5.0
Indiana Jones and the Last Crusade (1989)                5.0
Iron Giant, The (1999)                                   5.0
From Russia with Love (1963)                             5.0
Name: 1, dtype: float64

In [28]:
# 사용자가 기존에 감상한 영화는 추천 목록에서 제외한다.
def get_unseen_movies(ratings_matrix, userId):
    # userId로 입력받은 사용자의 모든 영화정보 추출하여 Series로 반환함. 
    # 반환된 user_rating 은 영화명(title)을 index로 가지는 Series 객체임. 
    user_rating = ratings_matrix.loc[userId,:]
    
    # user_rating이 0보다 크면 기존에 관람한 영화임. 대상 index를 추출하여 list 객체로 만듬
    already_seen = user_rating[ user_rating > 0].index.tolist()
    
    # 모든 영화명을 list 객체로 만듬. 
    movies_list = ratings_matrix.columns.tolist()
    
    # list comprehension으로 already_seen에 해당하는 movie는 movies_list에서 제외함. 
    unseen_list = [ movie for movie in movies_list if movie not in already_seen]
    
    return unseen_list

In [30]:
# 사용자에게 영화 추천하는 함수
def recomm_movie_by_userid(pred_df, userId, unseen_list, top_n=10):
    # 예측 평점 DataFrame에서 사용자id index와 unseen_list로 들어온 영화명 컬럼을 추출하여
    # 가장 예측 평점이 높은 순으로 정렬함. 
    # loc 함수를 사용해서 현재 사용자id와 사용자가 감상하지 않은 영화 리스트를 유사도가 가장 높은 순으로 추출한다.
    recomm_movies = pred_df.loc[userId, unseen_list].sort_values(ascending=False)[:top_n]
    return recomm_movies
    
# 사용자가 관람하지 않는 영화명 추출   
unseen_list = get_unseen_movies(ratings_matrix, 1)

# 아이템 기반의 인접 이웃 협업 필터링으로 영화 추천 
recomm_movies = recomm_movie_by_userid(ratings_pred_matrix, 1, unseen_list, top_n=10)

# 평점 데이타를 DataFrame으로 생성. 
recomm_movies = pd.DataFrame(data=recomm_movies.values,index=recomm_movies.index,columns=['pred_score'])
recomm_movies

,pred_score
title,
"Godfather: Part II, The (1974)",3.769505
"Sixth Sense, The (1999)",3.689380
"Godfather, The (1972)",3.503021
Trainspotting (1996),3.460017
Die Hard (1988),3.435728
Ferris Bueller's Day Off (1986),3.396243
Blade Runner (1982),3.227809
Airplane! (1980),3.200097
Jaws (1975),3.170789
